<a href="https://colab.research.google.com/github/Pankaj-2003/Strategy_Performance_Python/blob/main/Quant_Analyst_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime , timedelta

## Dataframe columns
### Date (datetime64[ns]): The date and time of the trade
### Symbol (string): The ticker symbol of the traded security
### Side (string): Either 'buy' or 'sell'
### Size (float, optional): The number of shares traded (default to 1 if not provided)
### Price (float): The price at which the trade was executed

In [2]:
def getTickerPrice(ticker: str, date: datetime) -> float:
    # This function returns the price of the security 'ticker' at the given 'date'
    # For the purpose of this exercise, assume it returns a random number
    return random.uniform(1, 100)  # Example implementation

In [ ]:
metrics = ["Sharpe" , "max-drawdown" ,  "profit factor" , "total net profit" , "Percent Profitable" , "average win\loss Ratio" , "Expectancy" , "Recovery factor" , "average r multiple" , "ROI"]

In [3]:
def calculate_sharpe(daily_returns, risk_free_rate=0.01):
    average_daily_return = daily_returns.mean()
    std_daily_return = daily_returns.std()

    # Avoid division by zero
    if std_daily_return == 0:
        return np.inf

    sharpe_ratio = (average_daily_return - risk_free_rate) / std_daily_return
    return sharpe_ratio

In [6]:
def calculate_max_drawdown(daily_returns):
    cumulative_return = (1 + daily_returns).cumprod()
    cumulative_max = cumulative_return.cummax()
    drawdown = cumulative_return - cumulative_max
    max_drawdown = drawdown.min()

    return max_drawdown

In [8]:
def calculate_profit_factor(trades):
    winning_trades = trades[trades['Return'] > 0]
    losing_trades = trades[trades['Return'] < 0]

    total_profit = winning_trades['Return'].sum()
    total_loss = -losing_trades['Return'].sum()  # Negate to get positive loss value

    if total_loss == 0:
        return np.inf if total_profit > 0 else 0  # edge case where there are no losses

    profit_factor = total_profit / total_loss
    return profit_factor

In [9]:
def percent_profitable(trades):
  winning_trades = trades[trades["Returns"] > 0]
  total_trades = len(trades)
  percent_profitable = (len(winning_trades) / total_trades) * 100
  return percent_profitable

In [12]:
def average_win_loss_ratio(trades):
  average_win = trades[trades["Return"] > 0]["Price"].mean()
  average_loss = trades[trades["Return"] < 0]["Price"].mean()
  average_win_loss_ratio = average_win / average_loss
  return average_win_loss_ratio

In [ ]:
from _typeshed import ProfileFunction
def trade_perf(trades : pd.dataframe) -> pd.series:
  # if dataframe is empty
  if trades.empty:
    return pd.Series({
            'Sharpe': 0,
            'max-drawdown': 0,
            'profit factor': 0,
            'total net profit': 0,
            'Percent Profitable': 0,
            'average win\loss Ratio': 0,
            'Expectancy': 0,
            'Recovery factor': 0,
            'average r multiple': 0,
            'ROI': 0})

  # handling missing values

  trades['Size'] = trades['Size'].fillna(1)
  trades = trades[trades['Symbol'].notna()]
  trades = trades[trades['Side'].notna()]

  # seperate way to fill price
  def fill_missing_prices(row):
    if pd.isna(row['Price']):
        row['Price'] = getTickerPrice(row['Symbol'], row['Date'])
    return row
  trades = trades.apply(fill_missing_prices, axis=1)




  # calculating returns
  def calculate_return(row):
    current_price = getTickerPrice(row['Symbol'], row['Date'])
    if row['Side'] == 'buy':
        return (current_price - row['Price']) * row['Size']
    else:
        return (row['Price'] - current_price) * row['Size']

  trades['Return'] = trades.apply(calculate_return, axis=1)  # Applying the function to each row of the df

  daily_returns = trades.groupby('Date')['Return'].sum() # daily return for sharpe



  # 1- calculating metrics

  sharpe_ratio = calculate_sharpe(daily_returns)

  # 2 - max_drawdown
  maxdd = calculate_max_drawdown(daily_returns)

  # 3 - profit factor
  profit_factor = calculate_profit_factor(trades)

  # 4 - total net profit
  total_net_profit = trades['Return'].sum()

  # 5 - percent profitable
  percent_profitable = percent_profitable(trades)

  # 6 - average win/loss ratio
  average_win_loss_ratio = average_win_loss_ratio(trades)

  # 7 - expectancy


















In [4]:
symbols = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA']
sides = ['buy', 'sell']
base_date = datetime(2024, 7, 15)
dates = [base_date + timedelta(days=i//4) for i in range(20)]
data = {
    'Date': dates,
    'Symbol': [random.choice(symbols) for _ in range(20)],
    'Side': [random.choice(sides) for _ in range(20)],
    'Size': [random.randint(1, 100) for _ in range(20)],
    'Price': [random.uniform(50, 300) for _ in range(20)],
}
trades = pd.DataFrame(data)




In [5]:
# Introduce missing values randomly
for col in ['Symbol', 'Size', 'Price']:
    trades.loc[random.sample(range(len(trades)), k=5), col] = np.nan  # Set 5 random entries in each column to NaN

In [ ]:
daily_returns = trades.groupby('Date')['Return'].sum()

In [7]:
trades

,Date,Symbol,Side,Size,Price
0,2024-07-15,AMZN,buy,NaN,NaN
1,2024-07-15,TSLA,sell,79.0,238.047256
2,2024-07-15,GOOGL,sell,84.0,247.301628
3,2024-07-15,MSFT,sell,43.0,NaN
4,2024-07-16,AAPL,sell,NaN,297.821150
5,2024-07-16,NaN,buy,NaN,NaN
6,2024-07-16,GOOGL,sell,36.0,288.395024
7,2024-07-16,AAPL,sell,64.0,259.690611
8,2024-07-17,AAPL,sell,27.0,252.235973
9,2024-07-17,MSFT,sell,47.0,177.192309


In [11]:
trades[trades["Price"] > 100]["Price"]

1     238.047256
2     247.301628
4     297.821150
6     288.395024
7     259.690611
8     252.235973
9     177.192309
10    116.528265
12    146.759882
13    116.519438
16    232.398059
17    201.473248
18    114.494575
19    295.867129
Name: Price, dtype: float64